In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize

In [3]:
# load data frame from csv
routes_df = pd.read_csv("routes_df.csv", sep='\t').drop('Unnamed: 0', axis=1)
routes_df.head()

,id,Aid,Alpine,Boulder,Ice,Mixed,Sport,TR,Trad,height,...,area_World WallaAreas,area_Write-off RockaAreas,area_Wu Wei WallaAreas,area_Wu-Tang WallaAreas,area_XY Crag (Chromosome Cracks)aAreas,area_Zappa WallaAreas,area_Zeke's Trail BouldersaAreas,area_Zeke's WallaAreas,area_Zig Zag WallaAreas,num_reviews
0,0,0,0,0,0,0,0,0,1,41.0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,1,41.0,...,0,0,0,0,0,0,0,0,0,1
2,2,0,0,0,0,0,0,1,1,35.0,...,0,0,0,0,0,0,0,0,0,4
3,3,0,0,0,0,0,0,1,1,85.0,...,0,0,0,0,0,0,0,0,0,4
4,4,0,0,0,0,0,0,1,1,30.0,...,0,0,0,0,0,0,0,0,0,4


In [4]:
route_id = routes_df['id']
routes_df = routes_df.drop('id', axis=1)

In [5]:
routes_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727 entries, 0 to 3726
Columns: 701 entries, Aid to num_reviews
dtypes: float64(2), int64(699)
memory usage: 19.9 MB


### Normalize data

In [7]:
items_mat = np.array(list(routes_df.values)).astype(float)

In [9]:
items_mat_norm = normalize(items_mat)

### get similiar routes

In [10]:
cos_sim = 1-pairwise_distances(items_mat, metric="cosine")
n = 10
index = 956 # id for Godzilla
arr = cos_sim[index]
similar_routes = np.asarray(route_id)[arr.argsort()[-(n+1):][::-1][1:]]
similar_routes

array([2086,  554,   44, 2658, 1705, 1200, 1244, 3627,  199, 1163])